# Description of the Notebook

This notebook is dedicated to fine-tuning the YOLO NAS model. We start by loading the `yolo_nas-l` model. The training dataset is organized as follows:

- **Training Images**: Stored in the `./data/train/train_img` directory.
- **Training Labels**: Located in the `./data/train/trainyolo` folder.

The model undergoes training for a total of **100 epochs**. Post each epoch, the following actions are taken:

1. **Model Checkpoint**: The state of the model is saved in the `./data/train/checkpoints` folder.
2. **Log File Creation**: A log file is generated and stored in the `./data/train/checkpoints` directory.

These log files are crucial for evaluating the training performance. They enable the plotting of various metrics to assess the model's learning curve, such as:

- **Training Loss vs. Epoch**
- **Validation Loss vs. Epoch**

This methodical approach ensures a detailed understanding and assessment of the model's performance during training.


In [2]:
from IPython.display import clear_output
import warnings
from super_gradients.training import models
from train_NAS_utils import *
import os
import torch
warnings.filterwarnings('ignore')
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

The console stream is logged into C:\Users\Admin\sg_logs\console.log


[2023-12-03 23:01:24] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-12-03 23:01:25] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-12-03 23:01:25,117] torch.distributed.elastic.multiprocessing.redirects: [WARNING] NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-12-03 23:01:25] INFO - utils.py - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-12-03 23:01:25] INFO - utils.py - NumExpr defaulting to 8 threads.
[2023-12-03 23:01:33] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-12-03 23:01:33] WARNING - export.py - Failed to import pytorch_quantization
[2023-12-03 23:01:33] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-12-03 23:01:33] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernel

### Loading Base Model pretrained with COC dataset

In [3]:
# options : 'yolo_nas_l' ,'yolo_nas_m','yolo_nas_s'
MODEL_ARCH = 'yolo_nas_l'
model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

[2023-12-03 22:58:38] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-12-03 22:58:39] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


Indexing dataset annotations: 100%|██████████| 2/2 [00:00<00:00, 1000.07it/s]


[2023-12-03 23:01:35] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-12-03 23:01:35] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


### Applying train transformations

In [4]:
train_data.dataset.transforms

[DetectionMosaic('additional_samples_count': 3, 'non_empty_targets': False, 'prob': 1.0, 'input_dim': (640, 640), 'enable_mosaic': True, 'border_value': 114),
 DetectionRandomAffine('additional_samples_count': 0, 'non_empty_targets': False, 'degrees': 10.0, 'translate': 0.1, 'scale': [0.1, 2], 'shear': 2.0, 'target_size': (640, 640), 'enable': True, 'filter_box_candidates': True, 'wh_thr': 2, 'ar_thr': 20, 'area_thr': 0.1, 'border_value': 114),
 DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': (640, 640), 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114),
 DetectionHSV('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30, 'bgr_channels': (0, 1, 2), '_additional_channels_warned': False),
 DetectionHorizontalFlip('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 0.5),
 DetectionPaddedRescale('additional_samples_count': 0, 'non_em

### Loading the pretrained model

In [5]:
model = models.get('yolo_nas_l',
                   num_classes= len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )


[2023-12-03 23:01:36] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-12-03 23:01:37] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


### Trainign Hyperparameters

These parameters can be changed to improve the performance of the model.

In [6]:
train_params = {
    'silent_mode': False,
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 50,
    "mixed_precision": False,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls = len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Start Finetuning

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)